# Person Directory

## Objective

 This script builds a Person Directory by enrolling multiple faces for each person from a directory where each subfolder represents a different person. It creates a person for each subfolder and adds all face images within that subfolder to the created person.

| Enrollment | Searching |
| :-: | :-: |
| ![PD_enrollment.png](https://media.githubusercontent.com/media/Azure-Samples/azure-ai-content-understanding-python/refs/heads/zhizho/face/data/face/PD_enrollment.png) | ![PD_searching.png](https://media.githubusercontent.com/media/Azure-Samples/azure-ai-content-understanding-python/refs/heads/zhizho/face/data/face/PD_searching.png) |

## Create Azure content understanding face client
>The [AzureContentUnderstandingFaceClient](../python/content_understanding_face_client.py) is utility Class which contain the functions to interact with the Content Understanding face server. Before Content Understanding SDK release, we can regard it as a lightweight SDK. Fill the constant **AZURE_AI_ENDPOINT**, **AZURE_AI_API_VERSION**, **AZURE_AI_API_KEY** with the information from your Azure AI Service.

In [ ]:
import logging
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# import utility package from python samples root directory
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_face_client import AzureContentUnderstandingFaceClient

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingFaceClient(
    endpoint=os.getenv("AZURE_AI_ENDPOINT"),
    api_version=os.getenv("AZURE_AI_API_VERSION", "2025-05-01-preview"),
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/build_person_directory", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

## Build a Person Directory

In [ ]:
import os
import uuid
folder_path = "../data/face/enrollment_data"  # Replace with the path to your folder containing subfolders of images

# Create a person directory
person_directory_id = f"person_directory_id_{uuid.uuid4().hex[:8]}"
client.create_person_directory(person_directory_id)
print(f"Created person directory with ID: {person_directory_id}")

# Iterate through all subfolders in the folder_path
for subfolder_name in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder_name)
    if os.path.isdir(subfolder_path):
        person_name = subfolder_name
        # Add a person for each subfolder
        person = client.add_person(person_directory_id, tags={"name": person_name})
        print(f"Created person {person_name} with person_id: {person['personId']}")
        if person:
            # Iterate through all images in the subfolder
            for filename in os.listdir(subfolder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(subfolder_path, filename)
                    # Convert image to base64
                    image_data = AzureContentUnderstandingFaceClient.read_file_to_base64(image_path)
                    # Add a face to the Person Directory and associate it to the added person
                    face = client.add_face(person_directory_id, image_data, person['personId'])
                    if face:
                        print(f"Added face from {filename} with face_id: {face['faceId']} to person_id: {person['personId']}")
                    else:
                        print(f"Failed to add face from {filename} to person_id: {person['personId']}")

print("Done")

### Identifying person
Detect multiple faces in an image and identify each one by matching it against enrolled persons in a Person Directory.

In [ ]:
test_image_path = "../data/face/family.jpg"  # Path to the test image

# Detect faces in the test image
image_data = AzureContentUnderstandingFaceClient.read_file_to_base64(test_image_path)
detected_faces = client.detect_faces(data=image_data)
for face in detected_faces['detectedFaces']:
    identified_persons = client.identify_person(person_directory_id, image_data, face['boundingBox'])
    if identified_persons.get("personCandidates"):
        person = identified_persons["personCandidates"][0]
        name = person.get("tags", {}).get("name", "Unknown")
        print(f"Detected person: {name} with confidence: {person.get('confidence', 0)} at bounding box: {face['boundingBox']}")

print("Done")

### Adding and associating a new face
You can add an additional face to a Person Directory and associate it with an already enrolled person.

In [ ]:
new_face_image_path = "new_face_image_path" # The path to the face image you want to add.
existing_person_id = "existing_person_id" # The unique ID of the person to whom the face should be associated.

# Convert the new face image to base64
image_data = AzureContentUnderstandingFaceClient.read_file_to_base64(new_face_image_path)
# Add the new face to the person directory and associate it with the existing person
face = client.add_face(person_directory_id, image_data, person['personId'])
if face:
    print(f"Added face from {new_face_image_path} with face_id: {face['faceId']} to person_id: {existing_person_id}")
else:
    print(f"Failed to add face from {new_face_image_path} to person_id: {existing_person_id}")

### Associating a list of already enrolled faces

You can associate a list of already enrolled faces in the Person Directory with their respective persons. This can be useful when you have existing face IDs and want to associate them to specific persons in the directory.

In [ ]:
existing_person_id = "existing_person_id"  # The unique ID of the person to whom the face should be associated.
existing_face_id_list = ["existing_face_id_1", "existing_face_id_2"]  # The list of face IDs to be associated.

# Associate the existing face IDs with the existing person
client.update_person(person_directory_id, existing_person_id, face_ids=existing_face_id_list)

### Associating and disassociating a face from a person
You can associate or disassociate a face from a person in the Person Directory. Associating a face links it to a specific person, while disassociating removes this link.

In [ ]:
existing_face_id = "existing_face_id"  # The unique ID of the face.

# Remove the association of the existing face ID from the person
client.update_face(person_directory_id, existing_face_id, person_id="") # The person_id is set to "" to remove the association
print(f"Removed association of face_id: {existing_face_id} from the existing person_id")
print(client.get_face(person_directory_id, existing_face_id)) # This will return the face information without the person association

# Associate the existing face ID with a person
existing_person_id = "existing_person_id"  # The unique ID of the person to be associated with the face.
client.update_face(person_directory_id, existing_face_id, person_id=existing_person_id)
print(f"Associated face_id: {existing_face_id} with person_id: {existing_person_id}")
print(client.get_face(person_directory_id, existing_face_id)) # This will return the face information with the new person association

### Updating metadata (tags and descriptions)
You can add or update tags for individual persons, and both descriptions and tags for the overall Person Directory. These metadata fields help you better organize, filter, and manage your directory.

In [ ]:
# Update the description and tags for the Person Directory
person_directory_description = "This is a sample person directory for managing faces."
person_directory_tags = {"project": "face_management", "version": "1.0"}

client.update_person_directory(
    person_directory_id,
    description=person_directory_description,
    tags=person_directory_tags
)
print(f"Updated Person Directory with description: '{person_directory_description}' and tags: {person_directory_tags}")
print(client.get_person_directory(person_directory_id)) # This will return the updated person directory information

# Update the tags for an individual person
existing_person_id = "existing_person_id"  # The unique ID of the person to update.
person_tags = {"role": "tester", "department": "engineering", "name": ""} # This will remove the name tag from the person.

client.update_person(
    person_directory_id,
    existing_person_id,
    tags=person_tags
)
print(f"Updated person with person_id: {existing_person_id} with tags: {person_tags}")
print(client.get_person(person_directory_id, existing_person_id)) # This will return the updated person information

### Deleting a face
You can also delete a specific face. Once the face is deleted, the association between the face and its associated person is removed.

In [ ]:
existing_face_id = "existing_face_id" # The unique ID of the face to delete.

client.delete_face(person_directory_id, existing_face_id)
print(f"Deleted face with face_id: {existing_face_id}")

### Deleting a person

When a person is deleted from the Person Directory, all the faces associated with that person remain in the Person Directory, but the association between the person and the faces is removed. This means the faces are no longer associated to any person in the Person Directory.

In [ ]:
existing_person_id = "existing_person_id"  # The unique ID of the person to delete.

client.delete_person(person_directory_id, existing_person_id)
print(f"Deleted person with person_id: {existing_person_id}")

### Deleting a person and their associated faces

To completely remove a person and all their associated faces from the Person Directory, you can delete the person along with their face associations. This operation ensures that no residual data related to the person remains in the directory.

In [ ]:
existing_person_id = "existing_person_id"  # The unique ID of the person to delete.

# Get the list of face IDs associated with the person
response = client.get_person(person_directory_id, existing_person_id)
face_ids = response.get('faceIds', [])

# Delete each face associated with the person
for face_id in face_ids:
    print(f"Deleting face with face_id: {face_id} from person_id: {existing_person_id}")
    client.delete_face(person_directory_id, face_id)

# Delete the person after deleting all associated faces
client.delete_person(person_directory_id, existing_person_id)
print(f"Deleted person with person_id: {existing_person_id} and all associated faces.")